In [1]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
import numpy as np
import faiss
from pathlib import Path
import os
import re
import pickle
from tqdm import tqdm

In [14]:
#Loading in markdown and chunks for testing

#This is for sanity testing, ignore this file v
markdown_path = Path("./postgre_chunks/accumulate.md")
markdown_text = markdown_path.read_text(encoding="utf-8")

#This is what we are using
chunks_path = Path("./postgre_chunks")
chunk_texts = []

id_count = 0

for file_path in chunks_path.glob("*.txt"):
    text = file_path.read_text(encoding="utf-8")
    chunk_texts.append({
        "id": id_count,
        "type" : "code",
        "content": text.strip()
    })
    id_count += 1

print(f"Loaded {len(chunk_texts)} chunks")


Loaded 4 chunks


In [17]:
emb_models = ["nomic-ai/nomic-embed-text-v1", "jinaai/jina-embeddings-v2-small-en"]
model = SentenceTransformer(
    emb_models[0],
    trust_remote_code=True
)

def embed_texts(texts: list[str]) -> np.ndarray:
    vectors = model.encode(texts, normalize_embeddings=True, convert_to_numpy=True)
    return vectors

# Do the embedding
all_texts = [chunk["content"] for chunk in chunk_texts]
all_embeddings = embed_texts(all_texts)

C:\Users\adika\.cache\huggingface\modules\transformers_modules\nomic-ai\nomic-bert-2048\7710840340a098cfb869c4f65e87cf2b1b70caca\modeling_hf_nomic_bert.py:119: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related

In [19]:
all_embeddings.shape

(4, 768)

In [22]:
import numpy as np
import json

In [23]:
vectors_np=np.array(all_embeddings).astype("float32")
dimension = vectors_np.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(vectors_np)

metadata_map = {i: chunk_texts[i] for i in range(len(chunk_texts))}

faiss.write_index(index, "rag_index.faiss")
with open("metadata_map.json","w") as f:
    json.dump(metadata_map, f)

print("embeddings stored in faiss, metadata saved.")    

embeddings stored in faiss, metadata saved.


In [29]:
# Load index
index = faiss.read_index("rag_index.faiss")
 
# Load metadata
with open("metadata_map.json", "r") as f:
    metadata_map = json.load(f)
 
# Embed a query
query = "How is card name generated?"
query_vec = model.encode([query], normalize_embeddings=True).astype("float32")
 
# Search top-5
D, I = index.search(query_vec, k=2)
 
# Lookup original chunks
for idx in I[0]:
    chunk = metadata_map[str(idx)]
    print(f"\n[{chunk['type']}] → {chunk['content']}")


[code] → Function getCardName Returns Char ():

   If Available credit-card Then
      Return '<INPUT TYPE="TEXT" CLASS="txtField" NAME="name" SIZE="20" MAXLENGTH="30" VALUE="' + credit-card.card-holder + '" tabindex="5">'. /* 030000 */
   Else
      Return '<INPUT TYPE="TEXT" CLASS="txtField" NAME="name" SIZE="20" MAXLENGTH="30" VALUE="' + v-addr-name + '" tabindex="5">'. /* 030000 */

End Function. /* getCardName() */

[code] → Function getBankName Returns Char ():

   Return '<INPUT TYPE="TEXT" CLASS="txtField" NAME="bankname" SIZE="20" MAXLENGTH="35" VALUE="' + v-bank-name + '" ' + v-disabled + ' tabindex="5">'. /* 010001 */

End Function. /* getCardExpiryYear() */
